# 크롤링과 스크래핑 원리 이해하기
유튜브 동영상 파이썬 활용편 <웹 스크래핑> 참조

## 웹페이지 정보 가져오는 requests 맛보기

In [ ]:
# pip install requests   # 아나콘다에 포함
# pip install beatifulsoup4   # 아나콘다에 포함
# pip install lxml   # 아나콘다에 포함

In [1]:
# requests 를 사용하여 웹 페이지 정보 가져오기

import requests

# 구글 웹페이지 정보를 저장
res = requests.get("http://google.com")
res.raise_for_status()   # 웹페이지를 정상적으로 불러오지 못하면 종료하는 기능

print(len(res.text))
# print(res.text)

with open("./mygoogle.html", "w", encoding="utf8") as f:
    f.write(res.text)

15244


In [10]:
# requests로 접속시 로봇으로 판단하여 접속이 안되는 경우 User-Agent 정보(사용자 OS, 웹브라우저 등 정보)를 
# requests의 get()함수 headers 옵션에 입력
# user agent string이라고 구글에 검색하면 나의 User-Agent 정보 확인 가능(What is my user agent? whatismybrower.com)

import requests

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
url = "https://www.ewha.ac.kr/ewha/index.do"

res = requests.get(url, headers = headers)
res.raise_for_status()

with open("./ewha.html", "w", encoding="utf8") as f:
    f.write(res.text)

## 네이버 웹툰 스크래핑하기(requests와 BeautifulSoup 사용)

In [23]:
import requests
from bs4 import BeautifulSoup

# 네이버 웹툰
url = "https://comic.naver.com/webtoon/weekday.nhn"
res = requests.get(url)
res.raise_for_status()   # 웹페이지를 가져오는데 문제가 있으면 종료

# pip install lxml
soup = BeautifulSoup(res.text, "lxml") # html 페이지 정보를 lxml parser로 파싱한 정보를 soup 객체에 저장

In [17]:
print(soup.title)
print(soup.title.get_text())
print(soup.a)  # soup 객체에서 처음 발견되는 a element 정보를 출력
print(soup.a.attrs)  # a element의 속성 정보를 출력
print(soup.a["href"]) # a element의 href 속성 정보

<title>네이버 만화 &gt; 요일별  웹툰 &gt; 전체웹툰</title>
네이버 만화 > 요일별  웹툰 > 전체웹툰
<a href="#menu" onclick="document.getElementById('menu').tabIndex=-1;document.getElementById('menu').focus();return false;"><span>메인 메뉴로 바로가기</span></a>
{'href': '#menu', 'onclick': "document.getElementById('menu').tabIndex=-1;document.getElementById('menu').focus();return false;"}
#menu


In [ ]:
soup.find("a", attrs={"class":"Nbtn_upload"})   # class 속성이 Nbtn_upload인 a element를 찾음
# soup.find(attrs={"class":"Nbtn_upload"})

In [18]:
# class 속성이 rank01인 li element를 찾은 후 li element 아래 첫 번째 a element의 text를 출력
rank1 = soup.find("li", attrs={"class":"rank01"})
print(rank1.a)
print(rank1.a.get_text())

<a href="/webtoon/detail?titleId=758037&amp;no=71" onclick="nclk_v2(event,'rnk*p.cont','758037','1')" title="참교육-71화">참교육-71화</a>
참교육-71화


In [27]:
# 네이버 웹툰 제목과 링크 추출하기
cartoons = soup.find_all("a", attrs={"class":"title"})

for cartoon in cartoons:
    title = cartoon.get_text()
    link = "https://comic.naver.com" + cartoon["href"]
    print(title, link)

참교육 https://comic.naver.com/webtoon/list?titleId=758037&weekday=mon
쇼미더럭키짱! https://comic.naver.com/webtoon/list?titleId=783054&weekday=mon
신의 탑 https://comic.naver.com/webtoon/list?titleId=183559&weekday=mon
뷰티풀 군바리 https://comic.naver.com/webtoon/list?titleId=648419&weekday=mon
퀘스트지상주의 https://comic.naver.com/webtoon/list?titleId=783052&weekday=mon
장씨세가 호위무사 https://comic.naver.com/webtoon/list?titleId=728750&weekday=mon
소녀의 세계 https://comic.naver.com/webtoon/list?titleId=654774&weekday=mon
백수세끼 https://comic.naver.com/webtoon/list?titleId=733074&weekday=mon
팔이피플 https://comic.naver.com/webtoon/list?titleId=774863&weekday=mon
앵무살수 https://comic.naver.com/webtoon/list?titleId=739115&weekday=mon
잔불의 기사 https://comic.naver.com/webtoon/list?titleId=768536&weekday=mon
만렙돌파 https://comic.naver.com/webtoon/list?titleId=759940&weekday=mon
리턴 투 플레이어 https://comic.naver.com/webtoon/list?titleId=752414&weekday=mon
매지컬 급식 https://comic.naver.com/webtoon/list?titleId=791062&weekday=mon
요리GO https

## 쿠팡 노트북 검색 정보 스크래핑하기(requests와 BeautifulSoup 사용)

In [2]:
import requests
import re
from bs4  import BeautifulSoup

# 쿠팡에서 노트북 검색시 첫 페이지 url (url 안에 page=1)
url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=1&rocketAll=false&searchIndexingToken=&backgroundColor="
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"}

res = requests.get(url, headers=headers)
res.raise_for_status()

soup = BeautifulSoup(res.text, "lxml")

items = soup.find_all("li", attrs={"class":re.compile("^search-product")})

print(items[0].find("div", attrs={"class":"name"}).get_text())

삼성전자 2021 노트북 플러스2 15.6, 퓨어 화이트, 셀러론, NVMe128GB, 8GB, WIN10 Pro, NT550XDA-K14AW


In [3]:
for item in items:
    # 로켓 배송 아닌 경우 제외
    rocket = item.find("span", attrs={"class":"badge rocket"})
    if not rocket:
        print("로켓 배송만 포함합니다")
        continue
        
    # 삼성전자 상품 제외
    name = item.find("div", attrs={"class":"name"}).get_text() # 제품명
    if "삼성전자" in name:
        print("삼성전자 제품 제외")
        continue

    price = item.find("strong", attrs={"class":"price-value"}).get_text() # 가격

    # 리뷰 50개 이상, 평점 4.2 이상만 조회
    rate = item.find("em", attrs={"class":"rating"}) # 평점 - 평점이 없는 경우가 있었음
    if rate:
        rate = rate.get_text()
    else:
        print("평점 없음")
        continue

    rate_cnt = item.find("span", attrs={"class":"rating-total-count"}) # 평점 수 (30)
    if rate_cnt:
        rate_cnt = rate_cnt.get_text()
        rate_cnt = rate_cnt[1:-1] # 평점 수 옆의 괄호 제거
    else:
        print("평점 수 없음")
        continue

    if float(rate) >= 4.2 and int(rate_cnt) >= 50:
        print(name.strip(), price, rate, rate_cnt, sep="☆")

삼성전자 제품 제외
로켓 배송만 포함합니다
LG전자 2020 울트라 PC 14, 화이트, 셀러론, 512GB, 8GB, WIN10 Home, 14U390-ME1TK☆526,000☆4.5☆421
삼성전자 제품 제외
로켓 배송만 포함합니다
로켓 배송만 포함합니다
베이직스 2021 베이직북13 2세대, 베이직 골드, BB1321FW, 셀러론, 256GB, 8GB, WIN10 Home☆379,000☆4.5☆61
HP 2021 노트북 15s, NVMe 256GB, 윈도우 포함, 8GB☆779,000☆5.0☆82
로켓 배송만 포함합니다
로켓 배송만 포함합니다
LG전자 2020 울트라 PC 15.6, 화이트, 펜티엄, 256GB, 4GB, WIN10 Home, 15U50N-LR26K☆550,180☆5.0☆236
Apple 2020 맥북 에어 13, 스페이스 그레이, M1, 256GB, 16GB, MAC OS, Z124000BL☆1,448,840☆5.0☆8234
LG전자 2020 울트라 PC 14, 화이트, 셀러론, 128GB, 4GB, WIN10 Home, 14U390-ME1TK☆417,000☆4.5☆421
로켓 배송만 포함합니다
로켓 배송만 포함합니다
LG전자 2020 울트라 PC 14, 화이트, 셀러론, 64GB, 4GB, WIN10 Home, 14U390-ME1TK☆379,000☆4.5☆114
LG전자 2021 울트라 PC 15.6 + 무선마우스 + 마우스패드 + HDMI케이블 + 노트북키스킨, 화이트, 코어i5 11세대, 256GB, 16GB, Free DOS, 15UD50P-GX50K☆868,000☆5.0☆406
레노버 2021 노트북 15.6, Sand, ideaPad Slim3-15ALC R5 82KU, 라이젠5, 256GB, 20GB, Free DOS☆698,000☆5.0☆367
LG전자 2020 울트라 PC 14, 화이트, 셀러론, 128GB, 4GB, WIN10 Home, 14U390-ME1TK☆417,000☆4.5☆421
평점 없음
한성컴퓨터 2020 

In [11]:
import requests
import re
from bs4  import BeautifulSoup

# 쿠팡 노트북 검색 결과를 페이지를 지정해서 출력
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"}

# 페이지 선택
def coupang(page):
    # 페이지 정보 출력
    print("##### 현재 페이지: " + str(page), "#####")
    # 페이지에 따라 url 변경
    url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=\
    scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={0}&\
    rocketAll=false&searchIndexingToken=&backgroundColor=".format(page)

    res = requests.get(url, headers=headers)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "lxml")

    items = soup.find_all("li", attrs={"class":re.compile("^search-product")})  # class 속성 값이 search-product로 시작하는 li element 찾기 

    for i in items:
        # 로켓 배송만 포함
        rocket = i.find("span", attrs={"class":"badge rocket"})
        if not rocket:
            print("로켓 배송만 포함합니다")
            continue
            
        # 삼성전자 상품 제외
        name = i.find("div", attrs={"class":"name"}).get_text() # 제품명
        if "삼성전자" in name:
            print("삼성전자 제품 제외")
            continue

        price = i.find("strong", attrs={"class":"price-value"}).get_text() # 가격

        # 리뷰 50개 이상, 평점 4.2 이상만 조회
        rate = i.find("em", attrs={"class":"rating"}) # 평점 - 평점이 없는 경우가 있었음
        if rate:
            rate = rate.get_text()
        else:
            print("평점 없음")
            continue

        rate_cnt = i.find("span", attrs={"class":"rating-total-count"}) # 평점 수 (30)
        if rate_cnt:
            rate_cnt = rate_cnt.get_text()[1:-1] # 평점 수 옆의 괄호 제거
        else:
            print("평점 수 없음")
            continue
        
        # 링크 추가
        link = i.find("a", attrs={"class":"search-product-link"})["href"]
        
        # 최종 출력
        if float(rate) >= 4.2 and int(rate_cnt) >= 50:
            print("-"*100)
            print(f"제품명: {name.strip()}")
            print(f"가격: {price}원")
            print(f"평점: {rate} ({rate_cnt})개")
            print(f"링크: {'https://www.coupang.com' + link}")
            print("-"*100)

for page in range(1, 3):
    coupang(page)

##### 현재 페이지: 1 #####
삼성전자 제품 제외
로켓 배송만 포함합니다
----------------------------------------------------------------------------------------------------
제품명: LG전자 2020 울트라 PC 14, 화이트, 셀러론, 512GB, 8GB, WIN10 Home, 14U390-ME1TK
가격: 526,000원
평점: 4.5 (421)개
링크: https://www.coupang.com/vp/products/4841548763?itemId=6257652932&vendorItemId=73553348809
----------------------------------------------------------------------------------------------------
삼성전자 제품 제외
로켓 배송만 포함합니다
로켓 배송만 포함합니다
----------------------------------------------------------------------------------------------------
제품명: 베이직스 2021 베이직북13 2세대, 베이직 골드, BB1321FW, 셀러론, 256GB, 8GB, WIN10 Home
가격: 379,000원
평점: 4.5 (61)개
링크: https://www.coupang.com/vp/products/6250688061?itemId=12668966123&vendorItemId=79936102915
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
제품명: HP 2021 노트북 15s, 

## 네이버 금융 시가총액 상위 기업 정보 스크래핑하기(requests와 BeautifulSoup 사용)
### 웹 스크래핑 내용을 파일에 저장

In [4]:
import csv
import requests
from bs4 import BeautifulSoup

url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="

filename = "시가총액1-200.csv"
f = open(filename, "w", encoding="cp949", newline="")   # 윈도우 사용자
# f = open(filename, "w", encoding="utf-8-sig", newline="")   # 윈도우 사용자
f = open(filename, "w", encoding="utf8", newline="")   # 맥 사용자

writer = csv.writer(f)

title = "N	종목명	현재가	전일비	등락률	액면가	시가총액	상장주식수	외국인비율	거래량	PER	ROE".split("\t")
writer.writerow(title)

for page in range(1, 5):    
    res = requests.get(url + str(page))
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    
    # 모든 행 지정: tr 태그
    data_rows = soup.find("table", attrs={"class":"type_2"}).find("tbody").find_all("tr")
    for row in data_rows:
        # 행별 모든 td 태그
        columns = row.find_all("td")

        # 특정 행(tr)은 단순히 줄 구분으로 td 태그가 1개만 존재하였음. 의미없는 데이터는 skip
        if len(columns) <=1:  
            continue
            
        data = [column.get_text().strip() for column in columns] # strip 불필요한 문자열 제거 - 지정하지 않는 경우 공백 제거
        writer.writerow(data)
        
f.close()

### 웹 스크래핑 내용을 데이터 프레임에 저장

In [14]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}

def naver_stock(number):
    
    url = f"https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page={number}"
    res = requests.get(url, headers = headers)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "lxml")
    
    # 모든 행 지정: tr 태그
    data_rows = soup.find("table", attrs={"class":"type_2"}).find("tbody").find_all("tr")
    
    stock_info = []
    
    for row in data_rows:
        # 행별 모든 td 태그
        col = row.find_all("td")

        # 특정 행(tr)은 단순히 줄 구분으로 td 태그가 1개만 존재하였음
        if len(col) <=1:
            continue
            
        data = [i.get_text().strip() for i in col] # strip 불필요한 문자열 제거 - 지정하지 않는 경우 공백 제거
        stock_info.append(data)
        # print(data)
    
    stock_info_df = pd.DataFrame(data = stock_info).iloc[:,:-1]
    stock_info_df.columns = title
    
    return stock_info_df


# 컬럼 제목 지정
title ="N	종목명	현재가	전일비	등락률	액면가	시가총액	상장주식수	외국인비율	거래량	PER	ROE".split("\t")

stock_final = pd.DataFrame()
for page in range(1,5):
     stock_final = pd.concat([stock_final, naver_stock(page)], axis=0)
        
stock_final

,N,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE
0,1,삼성전자,"70,200",200,+0.29%,100,"4,190,787","5,969,783",51.82,"8,907,965",12.15,13.92
1,2,LG에너지솔루션,"363,500","27,500",-7.03%,500,"850,590","234,000",3.30,"1,983,756",91.72,10.68
2,3,SK하이닉스,"116,000","1,000",-0.85%,"5,000","844,483","728,002",50.43,"3,152,057",10.49,9.53
3,4,NAVER,"329,000",0,0.00%,100,"539,721","164,049",54.78,"678,047",3.28,106.72
4,5,삼성바이오로직스,"799,000","12,000",+1.52%,"2,500","528,658","66,165",10.50,"51,447",135.93,8.21
...,...,...,...,...,...,...,...,...,...,...,...,...
45,196,영풍,"667,000","2,000",+0.30%,"5,000","12,286","1,842",4.06,577,10.36,4.03
46,197,LX인터내셔널,"31,250",500,+1.63%,"5,000","12,112","38,760",23.96,"594,386",4.51,23.72
47,198,대한전선,"1,405",10,-0.71%,100,"12,033","856,473",1.41,"4,806,625",39.03,0.92
48,199,명신산업,"22,700",900,-3.81%,500,"11,911","52,470",2.83,"234,831",-9.84,-67.39
